### Heroes Of Pymoli Data Analysis
* Observation 1

* Observation 2

* Observation 3
-----

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
df = pd.read_csv(file_to_load)

## Player Count

In [2]:
n_players = df.SN.nunique()
player_count_df = pd.DataFrame({"Total Players":[n_players]})
player_count_df

,Total Players
0,576


## Purchasing Analysis (Total)

In [3]:
# Number of unique items
n_items = len(df['Item Name'].unique())
# Average purchase price
av_price = df.Price.mean()
# Number of purchases
n_purchases = len(df)
# Total revenue
total_revenue = df.Price.sum()

pa_df = pd.DataFrame({"Number of Unique Items": [n_items],
                     "Average Purchase Price": [av_price],
                     "Number of Purchases": [n_purchases],
                     "Total Revenue": [total_revenue]})
pa_df["Total Revenue"] = pa_df["Total Revenue"].map('${:,.2f}'.format)
pa_df["Average Purchase Price"] = pa_df["Average Purchase Price"].map('${:,.2f}'.format)
pa_df

,Number of Unique Items,Average Purchase Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

In [4]:
# Group by player SN (we use this groupby object later)
sn_gp = df.groupby('SN')

# Calculate count and %-age of male, female, other/non-disclosed players
gendem = pd.DataFrame()
gendem['Total Count'] = sn_gp['Gender'].agg(max).value_counts()
gendem['Percentage of Players'] = gendem['Total Count'].apply(lambda x: x*100/n_players).map('{:.1f}%'.format)

# Display dataframe
gendem

,Total Count,Percentage of Players
Male,484,84.0%
Female,81,14.1%
Other / Non-Disclosed,11,1.9%



## Purchasing Analysis (Gender)

In [5]:
pag_df = pd.DataFrame()
gender_gp = df.groupby('Gender')

# Obtain purchase count
pag_df['Purchase Count'] = df.Gender.value_counts()

# Obtain avg purchase price
pag_df['Average Purchase Price'] = gender_gp.Price.mean().map('${:.2f}'.format)

# Obtain total purchase value
pag_df['Total Purchase Value'] = gender_gp.Price.sum().map('${:.2f}'.format)

# Obtain avg total purchase per person
pag_df['Avg Total Purchase per Person'] =\
sn_gp.agg({'Price': sum, 'Gender': max}).groupby('Gender').Price.mean().map('${:.2f}'.format)

pag_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Male,652,$3.02,$1967.64,$4.07
Female,113,$3.20,$361.94,$4.47
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

In [12]:
# Establish bins for ages
age_bins = [0, 10, 15, 20, 25, 30, 35, 40, float("inf")]
age_bin_labels = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

# Categorize the existing players using the age bins
agedem_s = pd.cut(df.Age, bins=age_bins, labels=age_bin_labels)

# Bin the original data frame by age
age_df = df
age_df['Age Range'] = agedem_s

# Create summary data frame to hold results
agedem_df = pd.DataFrame()

# Calculate counts and %-ages by age group
agedem_df['Total Count'] = age_df.groupby('SN')['Age Range'].agg(max).value_counts().sort_index()
agedem_df['Percentage of Players'] = agedem_df['Total Count'].map(lambda n: n*100/n_players)

# Round percentage column to two decimal points
agedem_df['Percentage of Players'] = agedem_df['Percentage of Players'].map('{:.2f}%'.format)

# Display table
agedem_df

,Total Count,Percentage of Players
10-14,41,7.12%
15-19,150,26.04%
20-24,232,40.28%
25-29,59,10.24%
30-34,37,6.42%
35-39,26,4.51%
40+,7,1.22%
<10,24,4.17%


## Purchasing Analysis (Age)

In [7]:
# Create a summary data frame to hold the results
agedem2_df = pd.DataFrame()

# Run basis calculations
agedem2_df['Purchase Count'] = age_df['Age Range'].value_counts().sort_index()
agedem2_df['Average Purchase Price'] = age_df.groupby('Age Range').Price.mean()
agedem2_df['Total Purchase Value'] = age_df.groupby('Age Range').Price.sum()
agedem2_df['Avg Total Purchase per Person'] = age_df.groupby('SN').agg({'Age Range': max, 'Price': sum}).groupby('Age Range').mean()

# Give the data prettier formatting
agedem2_df['Average Purchase Price'] = agedem2_df['Average Purchase Price'].map('${:.2f}'.format)
agedem2_df['Total Purchase Value'] = agedem2_df['Total Purchase Value'].map('${:.2f}'.format)
agedem2_df['Avg Total Purchase per Person'] = agedem2_df['Avg Total Purchase per Person'].map('${:.2f}'.format)

# Display the summary data frame
agedem2_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
<10,32,$3.40,$108.96,$4.54
10-14,54,$2.90,$156.60,$3.82
15-19,200,$3.11,$621.56,$4.14
20-24,325,$3.02,$981.64,$4.23
25-29,77,$2.88,$221.42,$3.75
30-34,52,$2.99,$155.71,$4.21
35-39,33,$3.40,$112.35,$4.32
40+,7,$3.08,$21.53,$3.08


## Top Spenders

In [8]:
# Calculate purchase count, average purchase price, and total purchase value
# with top spender first
top_sn_df = sn_gp.Price.agg(['count','mean',sum]).sort_values('sum', ascending=False)

# Give nicer formatting
top_sn_df['mean'] = top_sn_df['mean'].map('${:.2f}'.format)
top_sn_df['sum'] = top_sn_df['sum'].map('${:.2f}'.format)
top_sn_df = top_sn_df.rename(columns={'count': 'Purchase Count',
                         'mean': 'Average Purchase Value',
                         'sum': 'Total Purchase Value'})

# Display top 5
top_sn_df.head(5)

,Purchase Count,Average Purchase Value,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

In [9]:
# Retrieve the Item ID, Item Name, and Item Price columns.
# Group by Item ID and Item Name
# Perform calculations to obtain purchase count, item price, and total purchase value
item_agg = df.groupby(['Item ID', 'Item Name']).Price.agg(['count', max, sum])

# Create a summary data frame to hold the results
# Sort the purchase count column in descending order
popitems_df = item_agg.sort_values('count', ascending=False)

# Give the displayed data cleaner formatting
popitems_df['max'] = popitems_df['max'].map('${:.2f}'.format)
popitems_df['sum'] = popitems_df['sum'].map('${:.2f}'.format)
popitems_df = popitems_df.rename(columns={'count': 'Purchase Count',
                           'max': 'Item Price',
                           'sum': 'Total Purchase Value'})

# Display a preview of the summary data frame
popitems_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

In [10]:
# Sort the above table by total purchase value in descending order
profitems_df = item_agg.sort_values('sum', ascending=False)

# Give the displayed data cleaner formatting
profitems_df['max'] = profitems_df['max'].map('${:.2f}'.format)
profitems_df['sum'] = profitems_df['sum'].map('${:.2f}'.format)
profitems_df = profitems_df.rename(columns={'count': 'Purchase Count',
                           'max': 'Item Price',
                           'sum': 'Total Purchase Value'})

# Display a preview of the summary data frame
profitems_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
